## Importing dependencies & defining global variables

In [ ]:
import boto3
from pprint import pprint
import IPython.display as disp
from PIL import Image, ImageDraw
from pdf2image import convert_from_path

In [ ]:
exam_name = 'custom_exam'
exam_bucket = 'exams-martinig'
exams_dir = '../exams'
sample_file = f'{exams_dir}/custom_exam.pdf'
region = 'us-east-1'

## Defining functions

In [ ]:
def convert_pdf2jpeg(pdf_file, exam_name, output_dir):
    pages = convert_from_path(pdf_file)
    page_count = 0
    for page in pages:
        page_count+=1
        page.save(f'{output_dir}/{exam_name}_{page_count}pg.jpg', 'JPEG')

def load_image(filename):
    with open(filename, "rb") as imageFile:
      f = imageFile.read()
      return bytearray(f)
    
def save_pages(textract_response):
    pages=[]
    blocks = textract_response['Blocks']
    for block in blocks:
        if block['BlockType'] == 'PAGE':
           pages.append(block)
    return pages

def save_lines(textract_response):
    lines=[]
    blocks = textract_response['Blocks']
    for block in blocks:
        if block['BlockType'] == 'LINE':
           lines.append(block)
    return lines

def save_words(textract_response):
    words=[]
    blocks = textract_response['Blocks']
    for block in blocks:
        if block['BlockType'] == 'WORD':
           words.append(block)
    return words

def identify_questions(words):
    questions=[]
    for word in words:
        if word['Text'] == 'Questão':
            questions.append(word)
    return questions

def localize_questions(questions):
    questions_bboxes = []
    for question in questions:
        questions_bboxes.append(question['Geometry']['BoundingBox'])
    return questions_bboxes

def order_questions(questions):
    for question, next_question in zip(questions, questions[1:]):
        # verify if the difference is minimum
        if (((question['Left'] - next_question['Left'])*100)/question['Left']) < 25 or (((next_question['Left'])*100)/next_question['Left'] - question['Left']) < 25:
            questions.sort(key = lambda question: (question.get('Top')))
        else:
            questions.sort(key = lambda question: (question.get('Left'), question.get('Top')))

In [ ]:
def detect_page_style(width, questions_locations):
    middle_horizontal = width/2
    page_style = '' # centered, columns, broken_columns
    for question in questions_locations:
        if width * question['Left'] > middle_horizontal:
            # page has 2 columns
            if height * question['Top'] > (height * questions_locations[0]['Top'] * 0.10):
                # question is broken
                page_style = 'broken_columns'
            else:
                # question is not broken
                page_style = 'columns'
        else:
            page_style = 'centered'
    return page_style

def create_question_bbox(page_style, box, width, height, previous_question, next_question):
    middle_horizontal = width/2
    broken = False
    if page_style == 'centered':
        if box['Top'] <= previous_question['Top']:
            x0 = 0
            y0 = height*box['Top']
            x1 = width
            y1 = height*next_question['Top']
        else:
            # question is on the bottom of page
            x0 = 0
            y0 = height*box['Top']
            x1 = width
            y1 = height
    else:
        if width * box['Left'] < middle_horizontal:
            # question is on the left
            if box['Top'] <= previous_question['Top']:
                # question is on the top left
                if width * next_question['Left'] > middle_horizontal:
                    # next question is on the right
                    x0 = 0
                    y0 = height*box['Top']
                    x1 = width/2
                    y1 = height
                else:
                    x0 = 0
                    y0 = height*box['Top']
                    x1 = width/2
                    y1 = height*next_question['Top']
            else:
               # question is on the bottom left
                x0 = 0
                y0 = height*box['Top']
                x1 = width/2
                y1 = height
        else:
            # question is on the right
            if box['Top'] < next_question['Top']:
                # question is on the top right
                x0 = width/2
                y0 = height*box['Top']
                x1 = width
                y1 = height*next_question['Top']
            else:
                # question is on the bottom right
                x0 = width/2
                y0 = height*box['Top']
                x1 = width
                y1 = height
    return [x0, y0, x1, y1]

def create_broken_piece_bbox(width, height, next_question):
    x0 = width/2
    y0 = 0
    x1 = width
    y1 = height*next_question['Top']
    return [x0, y0, x1, y1]

def show_bbox(draw, box, color):
    return draw.rectangle(box, outline=color, width=4)
    
def cut_question(image, question_bbox):
    crop = image.crop(question_bbox)
    return crop

## Transforming exam PDF file to JPEG images

In [ ]:
convert_pdf2jpeg(sample_file, exam_name, exams_dir)

## Defining Cuts

In [ ]:
page = 1
test_image = f'{exams_dir}/{exam_name}_{page}pg.jpg'
disp.Image(test_image, width=400)

In [ ]:
image = Image.open(test_image)
width, height =image.size
draw = ImageDraw.Draw(image)
width, height

In [ ]:
textract = boto3.client('textract')

response = textract.detect_document_text(
    Document={
        'Bytes': load_image(test_image),
    }
)

In [ ]:
words = save_words(response)
len(words)

In [ ]:
questions = identify_questions(words)
len(questions)

In [ ]:
questions_locations = localize_questions(questions)
order_questions(questions_locations)
pprint(questions_locations)

In [ ]:
page_style = detect_page_style(width, questions_locations)
print(page_style)

In [ ]:
question_bbox = create_question_bbox(page_style, questions_locations[0], width, height, questions_locations[0], questions_locations[1])
show_bbox(draw, question_bbox, 'green')

In [ ]:
question_bbox = create_question_bbox(page_style, questions_locations[1], width, height, questions_locations[0], questions_locations[2])
show_bbox(draw, question_bbox, 'red')
if page_style == 'broken_columns':
    broken_question_bbox = create_broken_piece_bbox(width, height, questions_locations[2])
    show_bbox(draw, broken_question_bbox, 'red')

In [ ]:
question_bbox = create_question_bbox(page_style, questions_locations[2], width, height, questions_locations[1], questions_locations[2])
show_bbox(draw, question_bbox, 'yellow')

In [ ]:
image.save('questions_test.jpg', 'JPEG')

In [ ]:
cut = cut_question(image, create_question_bbox(page_style, questions_locations[0], width, height, questions_locations[0], questions_locations[1]))
cut.save('question_test.jpg', 'JPEG')